# В РАЗРАБОТКЕ

1. Проверить гипотезу: параметр `price` перчаток с параметром `case == Clutch Case`, в среднем сильнее зависит от показателя `wear_value`, чем с параметром `case == Glove Case, Operation Hydra Case`
2. Найти перчатки, у которых показатель `wear_value`: а) изменяется больше, чем у всех остальных; б) изменяется меньше, чем у всех остальных
3. Составить рейтинг перчаток, которые имеют сильную зависимость цены от `wear_value`, но которые можно чаще всего купить за стандартную цену. Посчитать среднюю прибыль перепродажи таких предметов за `n` дней.

In [4]:
import pandas as pd

In [9]:
df = pd.read_csv('data.csv')
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint (Well-Worn),131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave (Battle-Scarred),73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather (Battle-Scarred),94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web (Minimal Wear),609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid (Well-Worn),225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands (Well-Worn),135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands (Battle-Scarred),68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal (Field-Tested),100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega (Battle-Scarred),199.00,0.625256,1537803224,BS,Clutch Case


# Подготовка данных:

Уберем окончание значений `market_hash_name	`, которое указывает на `quality`:

In [10]:
for index, row in df.iterrows():
    df.loc[index, 'market_hash_name'] = row['market_hash_name'][: row['market_hash_name'].index('(')]
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint,131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


Значение `price` должно быть строго больше нуля.

Исключим из выборки строки, которые не удовлетворяют данному условию, если они имеются:

In [11]:
df = df[df['price'] > 0]
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint,131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


Значение `wear_value` должно лежать в интервале (0, 1)

Будем рассматривать только строки с корректным значением `wear_value`:

In [12]:
df = df[(df['wear_value'] > 0) & (df['wear_value'] < 1)]
df

,market_hash_name,price,wear_value,sold_at,quality,case
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
5,★ Hand Wraps | Leather,105.77,0.650310,1536427996,BS,"Glove Case, Operation Hydra Case"
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


## Найти перчатки, у которых показатель wear_value: а) изменяется больше, чем у всех остальных; б) изменяется меньше, чем у всех остальных

Для решения данной задачи необходимо найти дисперсию

В данном случае дельта степеней свободы равна 0


In [19]:
import numpy as np

Получим список всех `market_hash_name` и список всех `quality`

In [21]:
market_hash_names = df['market_hash_name'].unique()
qualities = df['quality'].unique()

Для каждой пары `market_hash_name` и `quality` найдем значение дисперсии:

In [22]:
import itertools

In [38]:
vars_info = []
for market_hash_name, quality in itertools.product(market_hash_names, qualities):
    vars_info.append({'market_hash_name': market_hash_name,
                      'quality': quality, 
                      'var': df[(df['market_hash_name'] == market_hash_name) & (df['quality'] == quality)]['wear_value'].var()})
vars_info = pd.DataFrame(vars_info)
vars_info

,market_hash_name,quality,var
0,★ Driver Gloves | Lunar Weave,BS,0.011211
1,★ Driver Gloves | Lunar Weave,MW,0.000182
2,★ Driver Gloves | Lunar Weave,WW,0.000302
3,★ Driver Gloves | Lunar Weave,FT,0.005614
4,★ Driver Gloves | Lunar Weave,FN,0.000003
...,...,...,...
235,★ Sport Gloves | Amphibious,BS,0.010551
236,★ Sport Gloves | Amphibious,MW,0.000384
237,★ Sport Gloves | Amphibious,WW,0.000361
238,★ Sport Gloves | Amphibious,FT,0.005724


Найдем максимальные и минимальные значения дисперсии для каждого значения `quality`:

In [108]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info[vars_info['quality'] == quality]['var'].max()
    min_var = vars_info[vars_info['quality'] == quality]['var'].min()
    maxs += vars_info[vars_info['var'] == max_var].to_dict(orient='records')
    mins += vars_info[vars_info['var'] == min_var].to_dict(orient='records')

Список перчаток каждого качества, у которых значение `wear_value` изменяется сильнее, чем у остальных:

In [109]:
pd.DataFrame(maxs)

,market_hash_name,quality,var
0,★ Sport Gloves | Vice,BS,0.012481
1,★ Hydra Gloves | Mangrove,MW,0.000557
2,★ Moto Gloves | Polygon,WW,0.000520
3,★ Hydra Gloves | Case Hardened,FT,0.006651
4,★ Sport Gloves | Vice,FN,0.000021


Список перчаток каждого качества, у которых значение `wear_value` изменяется слабее, чем у остальных:

In [110]:
pd.DataFrame(mins)

,market_hash_name,quality,var
0,★ Specialist Gloves | Mogul,BS,7.905624e-03
1,★ Driver Gloves | Racing Green,MW,1.715146e-04
2,★ Moto Gloves | Boom!,WW,1.611888e-04
3,★ Bloodhound Gloves | Bronzed,FT,4.228206e-03
4,★ Specialist Gloves | Crimson Kimono,FN,7.932144e-10


Получены значения дисперции для каждого вида перчаток на всей выборке. 
Проблема в том, что дисперсии считались для разного числа элементов. 

Посмотрим, сколько было совершено продаж перчаток каждого вида:

In [115]:
solds_count = []
for market_hash_name, quality in itertools.product(market_hash_names, qualities):
    solds_count.append({'market_hash_name': market_hash_name,
                      'quality': quality,
                      'total_solds': df[(df['market_hash_name'] == market_hash_name) & (df['quality'] == quality)]['wear_value'].count()})
solds_count = pd.DataFrame(solds_count)
solds_count.sort_values('total_solds')

,market_hash_name,quality,total_solds
119,★ Specialist Gloves | Buckshot,FN,0
234,★ Sport Gloves | Omega,FN,0
99,★ Hand Wraps | Cobalt Skulls,FN,0
54,★ Hydra Gloves | Mangrove,FN,0
24,★ Bloodhound Gloves | Snakebite,FN,0
...,...,...,...
208,★ Specialist Gloves | Forest DDPAT,FT,147
15,★ Driver Gloves | Imperial Plaid,BS,147
153,★ Driver Gloves | Diamondback,FT,147
78,★ Hand Wraps | Duct Tape,FT,147


Исключим из выборки элементы, у которых количество продаж равно `0` или `1`:

In [122]:
solds_count = solds_count[solds_count['total_solds'] > 1]
solds_count

,market_hash_name,quality,total_solds
0,★ Driver Gloves | Lunar Weave,BS,141
1,★ Driver Gloves | Lunar Weave,MW,145
2,★ Driver Gloves | Lunar Weave,WW,145
3,★ Driver Gloves | Lunar Weave,FT,135
4,★ Driver Gloves | Lunar Weave,FN,4
...,...,...,...
233,★ Sport Gloves | Omega,FT,139
235,★ Sport Gloves | Amphibious,BS,146
236,★ Sport Gloves | Amphibious,MW,89
237,★ Sport Gloves | Amphibious,WW,67


Найдем 1, 2, 3 квантили продаж для каждого качества:

In [202]:
quantiles = pd.DataFrame()

for quality in qualities:
    column_values = []
    for quantile_ in (0.25, 0.5, 0.75):
        column_values += list(solds_count[solds_count['quality'] == quality].quantile(quantile_).values)
    quantiles[quality] = column_values
quantiles.set_index(np.array(['25%', '50%', '75%']), inplace=True)

In [204]:
quantiles

,BS,MW,WW,FT,FN
25%,137.0,105.25,84.75,135.00,2.0
50%,141.0,134.00,117.00,139.00,3.0
75%,144.0,139.25,134.25,141.25,5.0


Получим значения дисперсий `wear_value` для каждого качества на каждом квантиле из уже найденных дисперсий:

In [205]:
vars_info_tmp = vars_info.copy()

In [212]:
vars_info_tmp = vars_info_tmp.merge(solds_count)


Первый квантиль:

In [229]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] <= quantiles[quality][0])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] <= quantiles[quality][0])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [230]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Arboreal,BS,0.011873,128
1,★ Moto Gloves | Spearmint,MW,0.000507,73
2,★ Hand Wraps | Arboreal,WW,0.000488,70
3,★ Driver Gloves | Crimson Weave,FT,0.006598,135
4,★ Sport Gloves | Vice,FN,0.000021,2


In [231]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Mogul,BS,7.905624e-03,137
1,★ Sport Gloves | Hedge Maze,MW,1.729535e-04,85
2,★ Sport Gloves | Vice,WW,1.806081e-04,69
3,★ Driver Gloves | Imperial Plaid,FT,4.698191e-03,130
4,★ Specialist Gloves | Crimson Kimono,FN,7.932144e-10,2


Второй квантиль:

In [232]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][0] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][1])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][0] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][1])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [233]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Sport Gloves | Vice,BS,0.012481,141
1,★ Driver Gloves | Diamondback,MW,0.000479,135
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hydra Gloves | Case Hardened,FT,0.006651,141
4,★ Sport Gloves | Arid,FN,0.000019,3


In [234]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Emerald Web,BS,8.008868e-03,143
1,★ Driver Gloves | Racing Green,MW,1.715146e-04,145
2,★ Moto Gloves | Boom!,WW,1.611888e-04,135
3,★ Bloodhound Gloves | Bronzed,FT,4.228206e-03,146
4,★ Driver Gloves | Racing Green,FN,4.186478e-07,4


Третий квантиль:

In [235]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][1] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][1] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [236]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Cobalt Skulls,BS,0.011728,146
1,★ Bloodhound Gloves | Guerrilla,MW,0.000399,146
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hand Wraps | Duct Tape,FT,0.006078,147
4,★ Bloodhound Gloves | Charred,FN,0.000013,5


In [237]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Foundation,BS,0.008430,147
1,★ Driver Gloves | Racing Green,MW,0.000172,145
2,★ Moto Gloves | Boom!,WW,0.000161,135
3,★ Bloodhound Gloves | Bronzed,FT,0.004228,146
4,★ Hand Wraps | Duct Tape,FN,0.000001,7


Четвертый квантиль:

In [238]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [239]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Cobalt Skulls,BS,0.011728,146
1,★ Bloodhound Gloves | Guerrilla,MW,0.000399,146
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hand Wraps | Duct Tape,FT,0.006078,147
4,★ Bloodhound Gloves | Charred,FN,0.000013,5


In [240]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Foundation,BS,0.008430,147
1,★ Driver Gloves | Racing Green,MW,0.000172,145
2,★ Moto Gloves | Boom!,WW,0.000161,135
3,★ Bloodhound Gloves | Bronzed,FT,0.004228,146
4,★ Hand Wraps | Duct Tape,FN,0.000001,7
